In [0]:
import keras
import cv2
import numpy as np
from keras.datasets import mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.models import Sequential
from tqdm import tqdm

Using TensorFlow backend.


In [0]:
batch_size = 64
epochs = 5
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
NUM_CLASSES = 10

In [0]:
def preprocess(imgs):
    
    processed = []
    
    for img in tqdm(imgs):
        processed.append(cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation = cv2.INTER_AREA))
    
    proccessed = np.array(processed)
    return proccessed.reshape(proccessed.shape[0], IMAGE_WIDTH, IMAGE_HEIGHT, 1)

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = preprocess(x_train)
x_test = preprocess(x_test)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 2s 0us/step


100%|██████████| 10000/10000 [00:00<00:00, 136802.31it/s]


x_train shape: (60000, 32, 32, 1)
60000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
conv_base = VGG16(input_shape = (IMAGE_HEIGHT,IMAGE_WIDTH, 1),
                         include_top = False, weights = None, classes=NUM_CLASSES)

In [0]:
conv_base.trainable = True

In [0]:
model = Sequential()

model.add(conv_base)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

In [0]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['categorical_accuracy'])

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
history = model.fit(x_train,y_train,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data = (x_test,y_test),
                  shuffle = True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 34s 570us/step - loss: 0.3083 - categorical_accuracy: 0.8974 - val_loss: 0.0609 - val_categorical_accuracy: 0.9812
Epoch 2/5
60000/60000 [==============================] - 27s 452us/step - loss: 0.0683 - categorical_accuracy: 0.9803 - val_loss: 0.0654 - val_categorical_accuracy: 0.9808
Epoch 3/5
60000/60000 [==============================] - 27s 450us/step - loss: 0.0487 - categorical_accuracy: 0.9863 - val_loss: 0.0424 - val_categorical_accuracy: 0.9877
Epoch 4/5
60000/60000 [==============================] - 27s 449us/step - loss: 0.0369 - categorical_accuracy: 0.9899 - val_loss: 0.0316 - val_categorical_accuracy: 0.9915
Epoch 5/5
60000/60000 [==============================] - 27s 452us/step - loss: 0.0326 - categorical_accuracy: 0.9911 - val_loss: 0.0338 - val_categorical_accuracy: 0.9894


In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 215us/step
Test loss: 0.033833027853025124
Test accuracy: 0.9894000291824341
